<a href="https://colab.research.google.com/github/JayanthGuru/TeluguSentenceRecognizer/blob/main/finalTOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries.

In [ ]:
# All the required imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from keras.models import load_model
from skimage.transform import rotate

#For RNN
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils import to_categorical
from keras.layers import Embedding, Bidirectional, LSTM, Dense


Navigating to Current Directory

In [ ]:
%cd '/content/drive/MyDrive/SentenceTOCR/final/'
version = 7

/content/drive/MyDrive/SentenceTOCR/final


In [ ]:
# Reads an image and returns an image as a 2d numpy array.
def read_image(path):
    return np.asarray(Image.open(path).convert('L'))


# Binarizes an image.The input is a 2d numpy array.
def Binarize_image(img):
    # Apply Gaussian filtering 
    filtered_img = cv2.GaussianBlur(img,(5,5),0)
    # Apply thresholding.Otsu's and binary.
    ret,Thresh_img = cv2.threshold(filtered_img,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    return Thresh_img
    

# Part-1: Splitting the images

 *Line level segmentation.*

In [ ]:
# For segmenting the image.

# Function for distance
def split_lines_d(img):
    x=[]
    start = 0
    stop = 0
    for i in range(len(img)-1):
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i-1]==0) == 0 :
            start = i
        #if i!=len(img)-1:
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i+1]==0) == 0 :
            stop = i
            x.append(stop-start)      
    return x

def min_dist(img):
  splited_dist = split_lines_d(img)
  sort_dist=sorted(splited_dist)
  diff=[]
  for i in range(len(sort_dist)-1):
    z=sort_dist[i+1]-sort_dist[i]
    diff.append([z,sort_dist[i]])
  diff2=sorted(diff)
  if diff2[-1][0]<sort_dist[0]:
    dist=sort_dist[0]-2
  else:
    dist=diff2[-1][1]
  return dist

def split_lines_k(img):
    x=[]
    start = 0
    stop = 0
    for i in range(len(img)-1):
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i-1]==0) == 0 :
            start = i
        #if i!=len(img)-1:
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i+1]==0) == 0 :
            stop = i
            x.append([stop,start])
    return x

def forming_lines(img):
  new=[]
  img2=split_lines_k(img)
  new.append([img2[0][1],img2[0][0]])
  for i in range(len(img2)-1):
    if img2[i+1][0]-img2[i+1][1]<=min_dist(img):
      if new[-1][0]==img2[i][1]:
        new[-1][1]=img2[i+1][0]
      else:
        new.append([img2[i][1],img2[i+1][0]])
    else:
      new.append([img2[i+1][1],img2[i+1][0]])
  return new
  

def split_lines(img):
  line_images=[]
  points=forming_lines(img)
  for i in points:
    start=i[0]
    stop=i[1]
    line_images.append(img[start:stop+1])
  return line_images

In [ ]:
def split_line(img):
    # Outputs an array containing no. of zeroes in each row
    def num_zero(img):
        temp_list = []
        for i in range(len(img)):
            temp_list.append(np.count_nonzero(img[i]==0))
        return np.array(temp_list)
    
    # Image to be trimmed at the top and bottom. 
    # arr argument is the array containing no. of zeroes in each row
    def trim_image(img,arr):
        start = 0
        stop = 0
        for i in range(len(arr)):
            if arr[i]!=0:
                start = i
                break
        for i in reversed(range(len(arr))):
            if arr[i] != 0:
                stop = i
                break
        return (img[start:stop+1],arr[start:stop+1])

    
    # Returns length of consecutive zeroes
    def length(arr,ind,ele):
        if ind != len(arr)-1:
            i = ind
            while i<=len(arr)-1 and arr[i]==ele:
                i+=1
                
            if i != ind:
                return i-ind
            else:
                return 1
        else:
            return 

    # Returns True if no word exists after the current row.
    def EOL(arr,ind,ele):
        temp = True
        for i in range(ind,len(arr)):
            if arr[i] != 0:
                temp = False
        return temp

    # Returns the length above which the image should be split
    # The argument is the array of no. of zeroes
    def limiting_length(arr):
        temp_list = []
        i=0
        while i< len(arr):
            temp=0
            if arr[i] == 0:
                temp = length(arr,i,0)
                temp_list.append(temp)
            i=i+temp+1

        # maximum difference between consecutive elements
        temp_list.sort()
        diff = 0
        lim_len = 0
        for i in range(1,len(temp_list)):
            temp = temp_list[i] - temp_list[i-1]
            if temp > diff:
                diff = temp
                lim_len = temp_list[i-1]
        return lim_len
    
    img = np.transpose(img)
    zeroes = num_zero(img)
    trim_img, zeroes = trim_image(img,zeroes) 
    limit_len = limiting_length(zeroes)
    splited_images = []
    start = 0
    stop = 0
    i=1
    
    while i<len(zeroes):
        if zeroes[i] != 0:
            start = i
            while i < len(zeroes):
                if zeroes[i]!=0:
                    i+=1
                else:
                    if length(zeroes,i,0) > limit_len:
                        break
                    else:
                        i+=length(zeroes,i,0)
            if length(zeroes,i,0) > limit_len or EOL(zeroes,i,0):
                stop = i
                temp_img = np.transpose(trim_img[start:stop])
                splited_images.append(temp_img)
                i+=limit_len
        else:
            i+=1
    return splited_images


def split_word(img1):
    img = np.transpose(img1)
    splitted_letters = []
    i=0
    while i < len(img):
        if np.count_nonzero(img[i]==0) != 0:
            start = i
            j=i+1
            while j < len(img):
                if np.count_nonzero(img[j]==0) == 0:
                    break
                j+=1
            temp_img = np.transpose(img[start:j])
            splitted_letters.append(temp_img)
            i=j
        else:
            i+=1
    return splitted_letters

# PART-2 : PREDICTION OF CHARACTER

In [ ]:
# Load the models for prediction of character.
from keras.models import load_model
from keras.models import model_from_json
import json
with open('./Training/model_chars_tccnn-l.json') as infile:
    json_char = json.load(infile)

CNN_model_1 = model_from_json(json_char)
CNN_model_1.load_weights('./Training/model_chars_tccnn-l.hdf5')

with open('./Training/model_v_g.json') as infile:
    json_char = json.load(infile)

CNN_model_2 = model_from_json(json_char)
CNN_model_2.load_weights('./Training/model_v_g_weights.hdf5')

#FUNCTION
def predict_from_char(char):
  img=char
  #img=cv2.transpose(img)
  img=cv2.resize(img,(32,32))
  img = np.asarray(img).reshape(1, 1, 32, 32)
  img = img.astype('float32')
  img = img/255.0
  chars=[]
  vg=[]
  out =  CNN_model_1.predict(img)
  chars.append(np.where(out==out.max())[1][:]+1)
  if np.where(out==out.max())[1][:]+1>=20 and np.where(out==out.max())[1][:]+1<=55:
    out2 = CNN_model_2.predict(img)
    vg.append(np.where(out2==out2.max())[1][:])
  else:
    vg.append(np.array([-1]))
  return chars[0][0],vg[0][0]

### *Closest Word from Dictonary. (using Get Close Method)*

In [ ]:
import difflib

def getClosetMatch(word, dictonary_words):
    closest_match = difflib.get_close_matches(word, dictonary_words, cutoff=0.5)
    if(len(closest_match) != 0):
      closest_match = closest_match[0]
    else:
      closest_match = dictonary_words[0]
    return closest_match

def getClosetMatchOfFinal(word, dictonary_words):
    closest_match = difflib.get_close_matches(word, dictonary_words, cutoff=0.5)
    if(len(closest_match) != 0):
      closest_match = closest_match[0]
    else:
      closest_match = dictonary_words[0]
    print("final_ans of getClose :",closest_match)
    return closest_match


# PART-3: RNN PART

In [ ]:
# Source Text
# Add all the text sentences into this.

'''
 The Sentence for training go into sentences.txt
'''

sampleText = open(os.getcwd() + "/assets/sentences.txt","r")
data = sampleText.read()
sentences = data.split("\n")
sampleText.close()
print("len of sentences file is", len(sentences))

subsetWords = list()
firstWordsOfLine = list() 

for i in sentences:
  wordsOfLine = i[:-1].split(" ")       #i[:-1] to not consider \n character.
  for j in range(len(wordsOfLine)):
    if wordsOfLine[j] != "":
      if(j == 0):
        firstWordsOfLine.append(wordsOfLine[j])
      subsetWords.append(wordsOfLine[j])

corpus = list(set(subsetWords))
print("length of corpus:",len(corpus))

len of sentences file is 193
length of corpus: 436


In [ ]:
import pickle
import string
import re
import keras




###########################################
RNN_model_name = "bilstm6_v7_language_model"
###########################################


# load the trained RNN model
model = keras.models.load_model(f'./Training/RNN_models/{RNN_model_name}.h5')

# load the tokenizer
if(os.path.isfile("./assets/tokenizer.pickle")):
    with open('./assets/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
else:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([data])
    # save the tokenizer to a file
    with open('./assets/tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)    


print("tokenizer", tokenizer.index_word)


# define a function to predict the next word and show all the predictions
def pred_next_word(input_string, model = model, tokenizer=tokenizer):
    # Split the input string into separate words
    words = input_string.lower().split()

    # Use the last few words as the partial sentence
    partial_sentence = " ".join(words[-model.input_shape[1]:])

    # Convert the partial sentence to a sequence of integers
    sequence = tokenizer.texts_to_sequences([partial_sentence])[0]

    # Pad the sequence to match the model's input shape
    sequence = pad_sequences([sequence], maxlen=model.input_shape[1])

    # Make the prediction
    prediction = model.predict(sequence)[0]

    # Get the index of the most likely next word
    next_word_index = np.argmax(prediction)

    # Convert the index back to a word
    next_word = tokenizer.index_word[next_word_index]

    return next_word

tokenizer {1: 'పులి', 2: 'ధర', 3: 'పండు', 4: 'చీమ', 5: 'జింక', 6: 'వంతు', 7: 'కొండ', 8: 'దారి', 9: 'నీటి', 10: 'గీత', 11: 'గల', 12: 'పైన', 13: 'రోజు', 14: 'ఫలం', 15: 'ఆది', 16: 'అంతం', 17: 'మంచు', 18: 'ప్రేమ', 19: 'బలి', 20: 'రండి', 21: 'పీట', 22: 'పూజ', 23: 'పంది', 24: 'చెడు', 25: 'ఖాకి', 26: 'బలపం', 27: 'పోరు', 28: 'సింహం', 29: 'గురువు', 30: 'ఆదేశం', 31: 'దూడ', 32: 'ఉమ', 33: 'కోటి', 34: 'చూడు', 35: 'సాటి', 36: 'చేప', 37: 'మాట', 38: 'లాలి', 39: 'రామ', 40: 'వంక', 41: 'నలుపు', 42: 'ఆనందం', 43: 'నిజం', 44: 'వంద', 45: 'పరీక్ష', 46: 'పరువు', 47: 'ఊత', 48: 'మంచం', 49: 'నోరు', 50: 'ఆట', 51: 'పాలు', 52: 'శని', 53: 'బంతి', 54: 'భూమి', 55: 'రాను', 56: 'వేడి', 57: 'కొన్ని', 58: 'సిద్ధంగా', 59: 'మధ్య', 60: 'మంచి', 61: 'ఋషి', 62: 'బహు', 63: 'దేవ', 64: 'ధీక్ష', 65: 'చేదు', 66: 'ఆకు', 67: 'తీపి', 68: 'పోక', 69: 'రాగి', 70: 'జలం', 71: 'లేరు', 72: 'పోటీ', 73: 'రారు', 74: 'అలా', 75: 'ఎలా', 76: 'గౌరి', 77: 'గుంపు', 78: 'గంజి', 79: 'నీరు', 80: 'పాపం', 81: 'ఆమె', 82: 'రక్ష', 83: 'తయారు', 84: 'గాడిద', 85: 

THE ORIGINAL MAIN FILE STARTS.

In [ ]:
f1 = open('./assets/char.txt','r')
temp_list = f1.readlines()
f1.close()
char = {}
i=0
for i in range(len(temp_list)):
    if i%2 == 0:
        char[(i/2)+1] = temp_list[i+1] 

f2 = open('./assets/vattu_gunintam.txt','r')
temp_list = f2.readlines()
f2.close()
vattu_gunintam={}
i=0
for i in range(len(temp_list)):
    if i%2 == 0:
        vattu_gunintam[(i/2)] = temp_list[i+1]
  
failed_cases = []
# -----------------------  INPUT --------------------------------------
senFile = open(f"./output/{RNN_model_name}_predicted_output.txt", "w")
noOfSamples = len(os.listdir(os.getcwd() + "/assets/dataset"))   
for senIdx in range(1, noOfSamples+1):
  inputImg = read_image("./assets/dataset/"+str(senIdx)+".jpg")
  inputImg = Binarize_image(inputImg)

  # ---------------------------------------------------------------------
  predicted_words = []      #let suppose its length is n
  recognized_words = []     #Then its len will be n
  final_verdict = []        #its len is n.

  word_images = []
  try:
    for line_image in split_lines(inputImg):
        for word_image in split_line(line_image):
            if(len(word_image[0]) < 30):                #non word images.
              continue
            word_images.append(word_image)
            ############################################
            # plt.imshow(word_image, cmap = "Greys")
            # plt.show()
            ############################################

    
    for wordIdx in range(len(word_images)):
        word_image = word_images[wordIdx]
        predicted_word = ""
        for letter_image in split_word(word_image):
            a,b = predict_from_char(letter_image)
            if b==-1:
                s = (char[a][:-2])
            else:
                s = (char[a][:-2]+vattu_gunintam[b][:-2])
            l = s.split("&#")
            for i in l[1:]:
              predicted_word += chr(int(i[:4]))
        
        predicted_words.append(predicted_word)                                    # this step is just to keep track of recognized words  
      
        # Word being passed to get closed match from corpus.
        if(wordIdx == 0):
          recognized_word =  getClosetMatch(predicted_word, firstWordsOfLine)
        else:
          recognized_word = getClosetMatch(predicted_word, corpus)

        recognized_words.append(recognized_word)                                   # this step is just to keep track of predicted words 


    ##############################################################
    # print("predicted Words :", predicted_words)
    # print("recognized Words :", recognized_words)
    ##############################################################


    final_verdict = recognized_words[0]+" "

    for idx in range(1,len(predicted_words)):
          recognized_word = recognized_words[idx]

          try:
            generated_word = pred_next_word(final_verdict)      ## The function pred_next_word is called here
            # generated_word = getClosetMatch(predicted_words[idx], [generated_word, recognized_word])                                          
          except:
            generated_word = "#"
          
          final_ans = generated_word
          final_verdict += final_ans+" "


    output = final_verdict

    print(senIdx, output[:-1])
    senFile.write(output[:-1]+"\n")
  except Exception as e:
    # print(e)
    failed_cases.append(senIdx)
    continue  
senFile.close()

print("No Of Case Failed",len(failed_cases))

1/1 [==============================] - 1s 1s/step
1 మంచం సోదరుడు
1/1 [==============================] - 0s 22ms/step
2 మామిడి కాయ
1/1 [==============================] - 0s 29ms/step
3 నది కారు
1/1 [==============================] - 0s 31ms/step
4 తుది ధర
1/1 [==============================] - 0s 30ms/step
5 సముద్ర పత్రికా
1/1 [==============================] - 0s 21ms/step
6 మంచి ఋషి
1/1 [==============================] - 0s 21ms/step
7 వైపు మొక్క
1/1 [==============================] - 0s 20ms/step
8 మంచం సోదరుడు
1/1 [==============================] - 0s 22ms/step
9 రాగి జలం
1/1 [==============================] - 0s 20ms/step
10 మంచి ఋషి
1/1 [==============================] - 0s 21ms/step
11 పులి గోరు
1/1 [==============================] - 0s 20ms/step
12 చెప్పండి పోర్ట్
1/1 [==============================] - 0s 21ms/step
13 రకం తీరం కళ
1/1 [==============================] - 0s 20ms/step
14 గౌరి పూజ
1/1 [==============================] - 0s 22ms/step
15 మంచి ఋషి
1/1 [==================

In [ ]:
# Final Sen Accuracy: 

sampleOuput = open(f"./output/{RNN_model_name}_predicted_output.txt", "r")
our_result = sampleOuput.readlines()
sampleOuput.close()

groundOutput = open("./assets/ground_truth.txt","r")
truth = groundOutput.readlines()
groundOutput.close()


# This is just to remove \n from every line
for i in range(len(our_result)):
  our_result[i] = our_result[i][:-1]


ground_truth = []
for i in range(len(truth)):
  if(i in failed_cases):
    continue
  else:
    truth[i] = truth[i].strip()
    ground_truth.append(truth[i])


correctly_recog_idx = []
incorrectly_recog_idx = []
acc = 0
for i in range(len(our_result)):
  if i in failed_cases:
    continue
  if(our_result[i] == ground_truth[i]):
    acc += 1
    correctly_recog_idx.append(i)
  else:
    incorrectly_recog_idx.append(i)

print("ACC is: "+ str(acc) +" out of "+ str(noOfSamples - len(failed_cases)))

print("len of correctly_recognized", len(correctly_recog_idx))

print("len of incorrectly_recognized", len(incorrectly_recog_idx))

ACC is: 72 out of 192
len of correctly_recognized 72
len of incorrectly_recognized 119


In [ ]:
import difflib

def calculateCER(source, target):
    diffs = list(difflib.ndiff(source, target))
    distance = sum(1 for diff in diffs if diff[0] != ' ')
    return distance

In [ ]:
if (os.path.isdir(f"./output/Results") == False):
    os.makedirs(f"./output/Results")

details = {}

details["two_word_acc"] = 0
details["three_word_acc"] = 0
details["four_word_acc"] = 0
details["two_first_word_acc"] = 0
details["two_second_word_acc"] = 0
details["three_first_word_acc"] = 0
details["three_second_word_acc"] = 0
details["three_third_word_acc"] = 0
details["four_first_word_acc"] = 0
details["four_second_word_acc"] = 0
details["four_third_word_acc"] = 0
details["four_fourth_word_acc"] = 0
details["total_two_word"] = 0
details["total_three_word"] = 0
details["total_four_word"] = 0
details["total_sentences"] = 0
details["accuracy"] = 0

details["two_word_CER"] = 0
details["three_word_CER"] = 0
details["four_word_CER"] = 0
details["extra_words_recog"] = 0
 
for i in range(len(our_result)):
    details["total_sentences"] += 1
    try:
      l = len(our_result[i].split(" "))
      if(l==2):
          details["total_two_word"] += 1
          if(i in incorrectly_recog_idx):
              if(len(ground_truth[i].split(" "))>=0 and our_result[i].split(" ")[0] == ground_truth[i].split(" ")[0]):
                details["two_first_word_acc"] += 1
              elif(len(ground_truth[i].split(" "))>=1 and our_result[i].split(" ")[1] == ground_truth[i].split(" ")[1]):
                details["two_second_word_acc"] += 1
              
              details["two_word_CER"] = calculateCER(our_result[i], ground_truth[i])
              
          else:
              details["two_word_acc"] += 1
      
      elif(l==3):
          details["total_three_word"] += 1
          if(i in incorrectly_recog_idx):
              if(len(ground_truth[i].split(" "))>=0 and our_result[i].split(" ")[0] == ground_truth[i].split(" ")[0]):
                details["three_first_word_acc"] += 1
              elif(len(ground_truth[i].split(" "))>=1 and our_result[i].split(" ")[1] == ground_truth[i].split(" ")[1]):
                details["three_second_word_acc"] += 1
              elif(len(ground_truth[i].split(" "))>=2 and our_result[i].split(" ")[2] == ground_truth[i].split(" ")[2]):
                details["three_third_word_acc"] += 1  

              details["three_word_CER"] = calculateCER(our_result[i], ground_truth[i])
          else:
            details["three_word_acc"] += 1

      elif(l==4):
          details["total_four_word"] += 1
          if(i in incorrectly_recog_idx):
              if(len(ground_truth[i].split(" "))>=0 and our_result[i].split(" ")[0] == ground_truth[i].split(" ")[0]):
                details["four_first_word_acc"] += 1
              elif(len(ground_truth[i].split(" "))>=1 and our_result[i].split(" ")[1] == ground_truth[i].split(" ")[1]):
                details["four_second_word_acc"] += 1
              elif(len(ground_truth[i].split(" "))>=2 and our_result[i].split(" ")[2] == ground_truth[i].split(" ")[2]):
                details["four_third_word_acc"] += 1  
              elif(len(ground_truth[i].split(" "))>=3 and our_result[i].split(" ")[3] == ground_truth[i].split(" ")[3]):
                details["four_fourth_word_acc"] += 1

              details["four_word_CER"] = calculateCER(our_result[i], ground_truth[i])
          else:
            details["four_word_acc"] += 1    
    except Exception as e:
      details["extra_words_recog"] += 1
      # print(i,e)
      # print(our_result[i]+" -> "+ground_truth[i])

details["accuracy"] = ((details["two_word_acc"] + details["three_word_acc"] + details["four_word_acc"]) / details["total_sentences"])*100



# # Dumping the details into {model_name}.pickle file.
with open(f'./output/Results/{RNN_model_name}.pickle', 'wb') as handle:
    pickle.dump(details, handle, protocol=pickle.HIGHEST_PROTOCOL)




12 list index out of range
రకం తీరం కళ -> అలా ఎలా
17 list index out of range
మంచి ఋషి పండు -> పాపం జింక
34 list index out of range
నౌక దీటు ధర -> పూలు దండ
48 list index out of range
చెలి సఖి దూడ -> రావణ పతకం
74 list index out of range
లేరు రండి పులి గోరు -> లక్ష రాయ హుండి


In [ ]:
for i in details:
  print(i, details[i])

import csv

with open(f'{RNN_model_name}_output.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in details.items():
        writer.writerow([key, value])

if os.path.isfile(f"./{RNN_model_name}_output.csv"):
  print("Successful")
else:
  print("Unsuccessful")

two_word_acc 45
three_word_acc 27
four_word_acc 1
two_first_word_acc 12
two_second_word_acc 0
three_first_word_acc 6
three_second_word_acc 0
three_third_word_acc 0
four_first_word_acc 3
four_second_word_acc 0
four_third_word_acc 0
four_fourth_word_acc 0
total_two_word 114
total_three_word 66
total_four_word 11
total_sentences 192
accuracy 38.02083333333333
two_word_CER 5
three_word_CER 17
four_word_CER 22
extra_words_recog 5
Successful


In [ ]:
Fcount = 0
count = 0
for i in range(len(our_result)):
  if len(our_result[i].split(" ")) == 2:
     count += 1
     print(our_result[i]+" -> "+ground_truth[i])
     if (our_result[i].split(" ")[0] == ground_truth[i].split(" ")[0] and our_result[i] != ground_truth[i]):
        Fcount += 1
        print("==========> "+our_result[i]+" -> "+ground_truth[i])

print(Fcount)
print(count)

మంచం సోదరుడు -> మంచి ఋషి
మామిడి కాయ -> పండు ఫలం
నది కారు -> ఆది అంతం
తుది ధర -> మంచు ప్రేమ
సముద్ర పత్రికా -> బహు బలి
మంచి ఋషి -> దేవ ధీక్ష
వైపు మొక్క -> చేదు ఆకు
మంచం సోదరుడు -> తీపి పోక
రాగి జలం -> రాగి జలం
మంచి ఋషి -> లేరు రండి
పులి గోరు -> పులి పీట
==========> పులి గోరు -> పులి పీట
చెప్పండి పోర్ట్ -> పోటీ రారు
గౌరి పూజ -> గౌరి పూజ
మంచి ఋషి -> పంది గుంపు
గంజి నీరు -> గంజి నీరు
చెడు చీమ -> చెడు చీమ
ఆమె వంతు -> ఆమె వంతు
మంచి ఋషి -> ఖాకి రక్ష
వాడు పీడ -> కొండ దారి
బలపం లక్ష -> బలపం తయారు
==========> బలపం లక్ష -> బలపం తయారు
గాడిద పోరు -> గాడిద పోరు
చివరి చేప -> సింహం ఆందోళం
గురువు ఆదేశం -> గురువు ఆదేశం
కొన్ని పుస్తకం -> భోగి సంబరం
మనసు తోక -> పులి గోరు
మంచి ఋషి -> చెలి సఖి
దూడ మూతి -> దూడ మూతి
మంచి ఋషి -> ధనం మూలం
మనసు తోక -> ఉమ వంతు
మంచి ఋషి -> కోటి చేతులు
మంచి ఋషి -> గేదె పాక
రాత్రి కొన్ని -> చూడు పాప
నోటీ నోటీసు -> నటి ఆశ
కొండ కరోన -> రైతు సంఘం
సామర్థ్యం పూర్తయింది -> సాటి పూజ
నటి ఆశ -> నీటి పంది
లాలి సేవ -> జాలి గుండి
చెలి సఖి -> వెండి తాడు
వాడు పీడ -> వాడు పీడ
మంచి ఋషి -> మేక బోను
న